In [3]:
#Designing data structure for Flights
tails = ['T1','T2','T3','T4','T5','T6']
#Designing data structure for Airports
airports = ['AUS','DAL','HOU']
#Designing data structure with format('Departure','Arrival') to flight time as value
flight_times = {('AUS','DAL'):50,('DAL','AUS'):50,('AUS','HOU'):45,('HOU','AUS'):45,('DAL','HOU'):65,('HOU','DAL'):65}
#Designing data structure for ground time at each airport
ground_time = {'AUS':25, 'DAL':30, 'HOU':35 }
#Designing data structure for the number of gates
gates = {'AUS':1,'DAL':1,'HOU':2}
#Designing data structure for the count of flights in air
in_air={'AUS':1,'DAL':1,'HOU':0}

#Defining function to get the Military Time by passing minutes since midnight as argument
def Military_time(msm):
    a=msm//60       #getting hour time
    b=msm%60        #getting minute time
    y=str(a).zfill(2)+str(b).zfill(2)       #coverting integer value to string and adding zeros to make military time
    return y
    
#Function define to calculate minute since midnight by taking military time as argument
def Min_midnight(mlt):
    conv_time=int(mlt[0:2])*60+int(mlt[2:])
    return conv_time

#Function define to calculate minimum departure time by taking arrival time and airport(arrived airport) into consideration
def min_dept_time(arr_time,arr_port):        #('0645','HOU')
    time=int(arr_time[0:2])*60+int(arr_time[2:])+ground_time[arr_port]
    return time

#Function define to calculate arrival time by taking minimum departure time and airport(flying to) into consideration
def arrival_time(dept_time,fly_port):         #('0720',AUS')
    time=int(dept_time[0:2])*60+int(dept_time[2:])+ flight_times[fly_port]
    return time

#Function define to monitor gates by taking the departing and arriving airports into consideration
def gate_term(dept,arr):             #('HOU','AUS')
    gates[dept]=gates[dept]-1        # Deducting 1 to indicate flight has departed
    in_air[arr]=in_air[arr]+1         # Adding 1 to indicate flight has arrived
        
#Function define to monitor flights in air by taking the departing and arriving airports into consideration
def in_air_term(dept,arr):             #('HOU','AUS')
    in_air[arr]=in_air[arr]-1
    gates[dept]=gates[dept]+1         
        
#Designing data structure for the first six flights
flight_schedule=[['T1','AUS','HOU','0600'],['T2','DAL','HOU','0600'],['T3','DAL','AUS','0625'],
        ['T4','HOU','DAL','0600'],['T5','HOU','AUS','0600'],['T6','HOU','DAL','0645']]

#Defined loop to calcuate the arrival_time of the first six flights
for i in range(len(flight_schedule)):
    flight_schedule[i].append(Min_midnight(flight_schedule[i][3]))
    flight_schedule[i].append(arrival_time((flight_schedule[i][3]),(flight_schedule[i][1],flight_schedule[i][2])))
    flight_schedule[i].append(Military_time(flight_schedule[i][5]))

#Define function to keep flights flying by monitoring the gate count and the flight in air
def depart(flight):
    if gates[flight]>=-1:
        if flight=='AUS':          #If AUS-->DAL or If AUS-->HOU
            if gates['DAL']<=2 and in_air['DAL']<=2:
                return('DAL')
            elif gates['HOU']<=3 and in_air['HOU']<=3:
                return('HOU')
            else:
                return "No Flight"       #returning No flights available string
        if flight=='DAL':         #If DAL-->AUS or If DAL-->HOU
            if gates['AUS']<=1 and in_air['AUS']<=1:
                return('AUS')
            elif gates['HOU']<=3 and in_air['HOU']<=3:
                return('HOU')
            else:
                return "No Flight"         #returning No flights available string
        if flight=='HOU':       #If HOU-->AUS or If HOU-->DAL
            if gates['AUS']<=1 and in_air['AUS']<=1:
                return('AUS')
            elif gates['DAL']<=2 and in_air['DAL']<=2:
                return('DAL')
            else:
                return "No Flight"           #returning No flights available string
    else:
        return"No Flight"


i=360   #Define starting time of flight as 0600

    
#Defined loop to keep on flights flying till 2200 and monitoring flights at each minute
while i<=1320:
    for j in range(len(flight_schedule)): #Using length of flight_schedule as variable length
        #Defined to indicate the arrival of flight at the airport
        if flight_schedule[j][5]==i and flight_schedule[j][4]>405:   #405 used due to already hardcoded 6 flights
            in_air_term(flight_schedule[j][1],flight_schedule[j][2])
        #Defined to indicate the departure of flight from airport
        if min_dept_time((flight_schedule[j][6]),flight_schedule[j][2])==i:
            flight=[]                               #Creating a flight data structure
            flight.append(flight_schedule[j][0])    #Adding the tail value to flight
            flight.append(flight_schedule[j][2])    #Adding the destination airport to flight
            k=depart(flight[1])
            if k=="No Flight":
                pass
            else:
                flight.append(k)
                flight.append(Military_time((min_dept_time((flight_schedule[j][6]),flight_schedule[j][2]))))
                #Adding the Minimum departure time from Minute since midnight
                flight.append(min_dept_time((flight_schedule[j][6]),flight_schedule[j][2]))
                #Adding the Arrival time from minute since midnight
                flight.append(arrival_time((flight[3]),(flight[2],flight[1])))
                #Checking condition for the estimated arrival time to be less than 2200
                if flight[5]<=1320:
                    #Adding the Arrival time in Military Time
                    flight.append(Military_time(flight[5]))
                    #Calling gate function to set up the count in gates
                    gate_term(flight[1],flight[2])
                    flight_schedule.append(flight)
            #Check condition if the arrival time is greater than 2200
                else:
                    pass

    #Incrementing each minute to keep flight flying        
    i+=1    

#Sorting the flight by tail number
flight_schedule.sort(key=lambda sublist: sublist[0])

#Designing data structure for header in the CSV file
flight_header=['tail_number','origin','destination','departure_time','arrival_time']


#Importing CSV Module
import csv
#Writing into a new CSV File
myfile = open('flight_schedule.csv','w',newline='')
wr = csv.writer(myfile,delimiter=',')
#Writing the flight header
wr.writerow(flight_header)
#Writing the individual flights
for c in flight_schedule:
    wr.writerow(c[0:4]+c[6:])
#Closing the File
myfile.close()      